In [1]:
import math

import numpy as np



class project1(object):



    def ZCB(self,r0, kappa, tau, sigma, theta):

        B = (1 - np.exp(-kappa * tau)) / kappa

        A = (theta - (sigma ** 2) / (2 * (kappa ** 2))) * (B - tau) - (sigma ** 2) / (4 * kappa) * (B ** 2)

        zero_price = np.exp(A) * np.exp(-r0 * B)

        return zero_price



    def Vasicek_paths(self,r0, kappa, theta, sigma, T, M, I):

        ''' Parameters

             ==========

            r0: instantaneous spot rate

            kappa: mean-reversion factor

            theta: long-run mean

            sigma: volatility factor

            T: final date/time horizon

            M: number of time steps

            I: number of paths'''

        dt = T / M

        r = np.zeros((M + 1, I), dtype=np.float64)

        r[0] = r0

        rh = np.zeros_like(r)

        rh[0] = r0

        np.random.seed(10)

        ran = np.random.standard_normal((M + 1, I))



        for t in range(1, M + 1):

            rh[t] = (rh[t - 1] + kappa * (theta -  rh[t - 1])

                     * dt + sigma * ran[t] * math.sqrt(dt))

            r[t] = rh[t]



        return r



    def zcb_price(self,r0, kappa, theta, sigma, T, M, I):

                dt = T / M

                r = self.Vasicek_paths(r0, kappa, theta, sigma, T, M, I)

                zcb = np.zeros((M + 1, I), dtype=np.float64)

                zcb[-1] = 1.0  # Face value of bond assigned to entire row

                for t in range(M, 0, -1):

                    zcb[t - 1] = zcb[t] * np.exp(-(r[t] + r[t - 1]) / 2 * dt)

                ZCB_cashflow=np.sum(zcb, axis=1) / I  #This will be average cash flow of ZCB equivalent to time step M

                return ZCB_cashflow[0]



    def ZCB_European_Price(self, r0, kappa, theta, sigma, T, M, I,k):

        #k stands for strike price of the option

        dt = T / M

        Year_Count=int(M/T) #this parameter used to discount from 5 year to 4 year to get terminal values of bond prices till 4 year

        n=M - Year_Count

        option_prince=0

        r = self.Vasicek_paths(r0, kappa, theta, sigma, T, M, I)

        zcb = np.zeros((M + 1, I), dtype=np.float)

        option_casflow =np.zeros((n+ 1, I), dtype=np.float64)

        zcb[-1] = 1000.0  # Face value of bond assigned to entire row

        for t in range(M,n, -1):

            #discounting 5 year bond face value till 4 year

            zcb[t - 1] = zcb[t] * np.exp(-(r[t] + r[t - 1]) / 2 * dt)



        option_casflow[-1]=np.maximum(zcb[n]-k,0)

        for t in range(n,0, -1):

            #discounting terminal pay off to T=0

            option_casflow[t - 1] = option_casflow[t] * np.exp(-(r[t] + r[t - 1]) / 2 * dt)



        option_price = np.sum(option_casflow[0],axis=0) / I  # taking the average of the discounted cash flow to get option price

        return(option_price)





    def Swap_Rate(self,r0, kappa, tau, sigma, theta):

        j=tau

        CF=np.zeros((j,),dtype=np.float64)

        for t in range(j,0,-1):

            #Calculating bond with different maturity using affine model

            #ZCB(self,r0, kappa, tau, sigma, theta):

            CF[t-1]=self.ZCB(0.08,0.5,t,0.03,0.05)

        j=tau

        swap_rate=float((1.0-CF[j-1])/(CF[j-1]+CF[j-2]+CF[j-3]+CF[j-4]+CF[j-5]))

        return swap_rate

In [2]:
if __name__ == "__main__":

    exer1=  project1()

    exer2 = project1()

    exer3 = project1()

    exer4 = project1()

    print('analytical price of zero coupon bond is :', round(exer1.ZCB(0.08,0.5,5,0.03,0.05),2))

    print('Simulated price of zero coupon bond is :', round(exer2.zcb_price(0.08, 0.5, 0.05,0.03,5,50,100),2))

    print('Swap rate value(in percent) is :', (round(exer1.Swap_Rate(0.08, 0.5, 5, 0.03, 0.05), 2))*100)

    print('European option price of zero coupon bond is :', round(exer4.ZCB_European_Price(0.08, 0.5, 0.05, 0.03, 5, 50, 100,900),2))

analytical price of zero coupon bond is : 0.74
Simulated price of zero coupon bond is : 0.75
Swap rate value(in percent) is : 6.0
European option price of zero coupon bond is : 44.32


C:\Users\AbhishekChaudhari\AppData\Local\Temp\ipykernel_37672\3043765016.py:143: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  CF=np.zeros((j,),dtype=np.float)
C:\Users\AbhishekChaudhari\AppData\Local\Temp\ipykernel_37672\3043765016.py:107: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  zcb = np.zeros((M + 1, I), dtype=np.float)
